In [1]:
import os
import uuid
from dotenv import load_dotenv
from pydantic_settings import BaseSettings
from pydantic import Field, ConfigDict
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
from typing_extensions import TypedDict, Annotated, Literal
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langgraph.store.memory import InMemoryStore
from langgraph.graph import StateGraph, add_messages, START, END
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
import pandas as pd

In [2]:
import os
from dotenv import load_dotenv
_ = load_dotenv()


In [3]:
class Settings(BaseSettings):
    model_config = ConfigDict(env_file='.env', env_file_encoding='utf-8',extra='ignore')
    model_name: str = Field(..., env='MODEL_NAME')
    groq_api_key: str = Field(..., env='GROQ_API_KEY')
    embedding_model_name: str = Field(..., env='EMBEDDING_MODEL_NAME')

In [4]:
settings = Settings()


In [5]:
llm = ChatGroq(
    model_name=settings.model_name,
    groq_api_key=settings.groq_api_key
)


In [6]:
embeddings = HuggingFaceEmbeddings(model_name=settings.embedding_model_name)
store = InMemoryStore(index={"embed": embeddings})

C:\Users\Manideep S\AppData\Local\Temp\ipykernel_24720\3585780073.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=settings.embedding_model_name)


In [7]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

In [8]:
Base = declarative_base()


In [9]:
class Question(Base):
    __tablename__ = 'questions'
    id = Column(Integer, primary_key=True)
    skill_code = Column(String)
    text = Column(String)
    answer = Column(String)

In [10]:
DATABASE_URL = os.getenv("DATABASE_URL")
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(bind=engine)

# Tools


In [11]:
@tool
def get_skill_questions(skill_code: str) -> list:
    """Fetch 5 filtered questions for a given skill from joined prompt/question tables."""
    with SessionLocal() as session:
        query = session.execute(f'''
            SELECT
                q.id,
                q.question_text AS text,
                q.correct_answer AS answer,
                q.passage,
                q.question_type,
                q.rationale,
                q.options,
                q.hint,
                q.prompt_id,
                p.name AS prompt_name
            FROM public.api_tempquestion q
            JOIN public.api_tempprompt p ON q.prompt_id = p.id
            WHERE p.name LIKE 'PID-M-GAT-{skill_code}%'
            	AND q.question_type = 'MCQ'
            ORDER BY p.name ASC
            LIMIT 5
        ''')
        result = query.mappings().all()
        return [dict(row) for row in result]


In [12]:
@tool
def write_results_to_excel(results: list[dict], filename: str = "quiz_results.xlsx") -> str:
    """Save results to an Excel file."""
    df = pd.DataFrame(results)
    df.to_excel(filename, index=False)
    return f"Results saved to {filename}"

# Nodes


In [13]:
class QuizState(TypedDict):
    messages: Annotated[list, add_messages]
    skill_selected: str
    questions: list
    current_question_index: int
    user_answers: list

In [14]:
def greet_and_select_skill(state: QuizState):
    content = (
        "Hello! 👋 Let's begin your math quiz! Please choose one skill to focus on:\n"
        "- LAT: Lines Angles and Triangles\n"
        "- CIR: Circles\n"
        "- AAV: Areas and Volumes\n"
        "- RTT: Right Triangles and Trigonometry"
    )
    return {"messages": [{"role": "user", "content": content}]}

In [15]:
def fetch_questions_node(state: QuizState):
    questions = get_skill_questions(state["skill_selected"])
    return {"questions": questions, "current_question_index": 0}

In [16]:
def ask_question_node(state: QuizState):
    q_index = state["current_question_index"]
    question = state["questions"][q_index]
    question_text = question["text"] if isinstance(question, dict) else question.text
    content = f"Question {q_index+1}: {question_text}"
    answer = input("Enter your answer: ")
    state["user_answers"].append(answer)
    return {"messages": state["messages"] + [{"role": "user", "content": content}]}

In [ ]:
def answer_input_node(state: QuizState):
    q_index = state["current_question_index"]
    question = state["questions"][q_index]
    answer = state["user_answers"][-1]
    correct_answer = question["answer"] if isinstance(question, dict) else question.answer
    is_correct = answer == correct_answer
    feedback = "Correct!" if is_correct else f"Incorrect! The correct answer is {correct_answer}."
    return {"messages": state["messages"] + [{"role": "user", "content": feedback}]}

In [17]:
def check_answer_node(state: QuizState):
    idx = state["current_question_index"]
    question = state["questions"][idx]
    question_text = question["text"] if isinstance(question, dict) else question.text
    correct_answer = question["answer"] if isinstance(question, dict) else question.answer

    # Get the latest user response (which should be the last assistant prompt + user reply pair)
    messages = state.get("messages", [])

    # Assume last entry is the user's response to the question
    user_response = ""
    for msg in reversed(messages):
        if isinstance(msg, dict) and msg.get("role") == "assistant":
            continue  # skip LLM message
        elif isinstance(msg, dict) and msg.get("role") == "user":
            user_response = msg.get("content")
            if isinstance(user_response, list):
                user_response = user_response[-1]  # fix accidental list wrap
            break

    if not isinstance(user_response, str):
        user_response = str(user_response)

    if isinstance(user_response, list):
        user_response = user_response[0]  # Take the first element if it's a list
    is_correct = user_response.strip().lower() == correct_answer[0].lower()

    result = {
        "question": question_text,
        "correct_answer": correct_answer,
        "user_answer": user_response,
        "is_correct": is_correct
    }

    updated_answers = state.get("user_answers", []) + [result]

    return {
        "messages": messages,
        "questions": state["questions"],
        "skill_selected": state["skill_selected"],
        "user_answers": updated_answers,
        "current_question_index": idx + 1
    }

In [18]:
def generate_summary_node(state: QuizState):
    correct_count = sum(ans["is_correct"] for ans in state["user_answers"])
    total = len(state["user_answers"])

    summary_prompt = f"""
You are a supportive assistant. The user completed a quiz.
Generate a motivational message for scoring {correct_count}/{total}.
Encourage improvement and celebrate effort.
"""
    result = llm.invoke([{"role": "system", "content": summary_prompt}])
    return {"messages": [{"role": "user", "content": result.content}]}

In [19]:
def save_to_excel_node(state: QuizState):
    # Use invoke instead of direct call due to LangChain tool protocol
    return_value = write_results_to_excel.invoke({
        "results": state["user_answers"],
        "filename": "quiz_results.xlsx"
    })
    return state

# Graph setup


In [ ]:
quiz_graph = StateGraph(QuizState)
quiz_graph.add_node("greet", greet_and_select_skill)
quiz_graph.add_node("fetch_questions", fetch_questions_node)
quiz_graph.add_node("ask_question", ask_question_node)
quiz_graph.add_node("take_answer_input", answer_input_node)
quiz_graph.add_node("check_answer", check_answer_node)
quiz_graph.add_node("generate_summary", generate_summary_node)
quiz_graph.add_node("save_results", save_to_excel_node)

In [21]:
# Edges
quiz_graph.add_edge(START, "greet")
quiz_graph.add_edge("greet", "fetch_questions")
quiz_graph.add_edge("fetch_questions", "ask_question")
quiz_graph.add_edge("ask_question", "check_answer")

In [22]:
def next_step(state: QuizState):
    if state["current_question_index"] < 5:
        return "ask_question"
    else:
        return "generate_summary"
quiz_graph.add_conditional_edges("check_answer", next_step)
quiz_graph.add_edge("generate_summary", "save_results")
quiz_graph.add_edge("save_results", END)

In [23]:
app = quiz_graph.compile()

In [24]:
# Visualize LangGraph nodes and edges
graph_structure = app.get_graph()

# Text-based graph in terminal
graph_structure.print_ascii()

# Mermaid diagram (for notebooks or Markdown-friendly renderers)
from IPython.display import Markdown
graph_structure.draw_mermaid()

   +-----------+     
   | __start__ |     
   +-----------+     
          *          
          *          
          *          
     +-------+       
     | greet |       
     +-------+       
          *          
          *          
          *          
+-----------------+  
| fetch_questions |  
+-----------------+  
          *          
          *          
          *          
  +--------------+   
  | ask_question |   
  +--------------+   
          *          
          *          
          *          
  +--------------+   
  | check_answer |   
  +--------------+   
          *          
          *          
          *          
    +---------+      
    | __end__ |      
    +---------+      


'---\nconfig:\n  flowchart:\n    curve: linear\n---\ngraph TD;\n\t__start__(<p>__start__</p>)\n\tgreet(greet)\n\tfetch_questions(fetch_questions)\n\task_question(ask_question)\n\tcheck_answer(check_answer)\n\tgenerate_summary(generate_summary)\n\tsave_results(save_results)\n\t__end__(<p>__end__</p>)\n\t__start__ --> greet;\n\task_question --> check_answer;\n\tfetch_questions --> ask_question;\n\tgreet --> fetch_questions;\n\tcheck_answer --> __end__;\n\tclassDef default fill:#f2f0ff,line-height:1.2\n\tclassDef first fill-opacity:0\n\tclassDef last fill:#bfb6fc\n'

In [25]:
example_input = {
    "skill_selected": "CIR",
    "messages": [],
    "user_answers": ['a','b','c','d','a']
}

In [26]:
state = QuizState()

print(state)


{}


In [27]:
response = app.invoke(example_input)

C:\Users\Manideep S\AppData\Local\Temp\ipykernel_24720\2605161915.py:2: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  questions = get_skill_questions(state["skill_selected"])


TypeError: string indices must be integers, not 'str'

In [ ]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Hello! 👋 Let's begin your math quiz! Please choose one skill to focus on:
- LAT: Lines Angles and Triangles
- CIR: Circles
- AAV: Areas and Volumes
- RTT: Right Triangles and Trigonometry
================================ Human Message =================================

Question 1: "The circumference of a circle is ##16\\pi##. What is its area?"
================================ Human Message =================================

Question 2: "If an inscribed angle in a circle measures ##(x+15)## degrees and its intercepted arc measures ##(3x+5)## degrees, what is the measure of the inscribed angle?"
================================ Human Message =================================

Question 3: "A chord in a circle subtends a central angle of ##60^{\\circ}## in a circle with radius ##6##. What is the length of the chord?"
================================ Human Message =================================

Question 4

In [29]:
if __name__ == "__main__":
    print("📚 Welcome to the Math Quiz CLI!")
    skill = input("Choose a skill [LAT / CIR / AAV / RTT]: ").strip().upper()

    state = QuizState()

    from langchain_core.messages import AIMessage, HumanMessage

    state = app.invoke(state)


    final_msg = state["messages"][-1]
    if isinstance(final_msg, (AIMessage, HumanMessage)):
        final_content = final_msg.content
    elif isinstance(final_msg, dict):
        final_content = final_msg.get("content", "")
    else:
        final_content = str(final_msg)

    print("\n📊 Quiz Summary:")
    print(final_content)

    # Optional: show full answers
    print("\n📝 Detailed Results:")
    for result in state["user_answers"]:
        print(f"Q: {result['question']}")
        print(f"✅ Correct: {result['correct_answer']} | ❌ Your Answer: {result['user_answer']}")
        print(f"🎯 Result: {'Correct' if result['is_correct'] else 'Incorrect'}\n")

📚 Welcome to the Math Quiz CLI!


KeyError: 'skill_selected'